In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.linear_model as skl
import pandas as pd
%matplotlib inline

In [3]:
# loading training/testing data
train = pd.read_csv("train.csv", parse_dates = ['purchase_date','release_date'])
test = pd.read_csv("test.csv",parse_dates = ['purchase_date','release_date'])
print(len(train))
print(len(test))
print(train.shape)
print(test.shape)

357
90
(357, 11)
(90, 10)


In [4]:
#print(train.columns)
#拼接原来的train和test,记作all_data，并记录test的编号
all_data = train.drop(['playtime_forever'],axis = 1)
all_data = pd.concat([all_data,test], axis=0)
all_data = all_data.reset_index(drop=True)
all_data.tail(100)
#print(all_data.shape)

,id,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews
347,347,False,20800.0,"Action,Adventure","Single-player,Multi-player,Co-op,Steam Trading...","Open World,Shooter,Action,Co-op,Multiplayer,Ta...",2018-02-16,2017-03-06,23338.0,7681.0
348,348,False,2000.0,"Adventure,Casual,Indie","Single-player,Steam Achievements,Steam Trading...","Adventure,Indie,Interactive Fiction,Casual,Bas...",2019-03-28,2013-04-25,263.0,63.0
349,349,False,4800.0,"Action,Indie,RPG","Single-player,Local Multi-Player,Local Co-op,S...","Pixel Graphics,Zombies,Action,Rogue-like,Survi...",2017-03-25,2016-07-21,3083.0,202.0
350,350,False,3700.0,Action,"Single-player,Multi-player,Co-op,Steam Achieve...","Quick-Time Events,Rome,Action,Hack and Slash,T...",2018-01-23,2014-10-10,11922.0,2303.0
351,351,False,5000.0,"Action,Adventure,Indie,RPG","Single-player,Steam Achievements,Full controll...","Indie,Adventure,Action,Action-Adventure,Explor...",2018-11-17,2017-05-05,106.0,23.0
352,352,False,8800.0,"Action,Simulation","Single-player,Multi-player,Online Multi-Player...","Simulation,Action,VR,Flight",2017-11-24,2016-12-20,150.0,91.0
353,353,False,6800.0,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Batman,Action,Open World,Superhero,Stealth,Adv...",2018-08-15,2015-06-23,19008.0,4849.0
354,354,False,8300.0,"Action,Adventure,RPG","Single-player,Steam Achievements,Full controll...","Action,Hack and Slash,Adventure,RPG,Open World...",2018-01-30,2015-11-05,5099.0,1719.0
355,355,False,6800.0,"Action,Adventure","Single-player,Steam Achievements,Full controll...","Action,FPS,Adventure,Shooter,Cyberpunk,Robots,...",2017-09-23,2016-06-03,718.0,159.0
356,356,False,10000.0,"Indie,Strategy","Single-player,Multi-player,Co-op,Shared/Split ...","Strategy,Turn-Based Strategy,Indie,Grand Strat...",2018-03-05,2013-12-05,915.0,102.0


In [5]:
all_data = all_data.fillna(all_data.mean())

#train['purchase_date']=train['purchase_date'].fillna("0")
all_data.isnull().any()   #此时日期仍有缺失

id                        False
is_free                   False
price                     False
genres                    False
categories                False
tags                      False
purchase_date              True
release_date              False
total_positive_reviews    False
total_negative_reviews    False
dtype: bool

In [6]:
#构造列--购买时间间隔
interval = []
for i in range(len(all_data)):
    purchase = all_data['purchase_date'][i]
    release = all_data['release_date'][i]
    diff = (purchase-release).days
    interval.append(diff)
all_data['purchase_interval'] = np.array(interval)
all_data = all_data.fillna(all_data.mean())
all_data.head(7)

,id,is_free,price,genres,categories,tags,purchase_date,release_date,total_positive_reviews,total_negative_reviews,purchase_interval
0,0,False,3700.0,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",2018-07-02,2013-12-10,372.000000,96.000000,1665.000000
1,1,True,0.0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",2016-11-26,2015-08-12,23.000000,0.000000,472.000000
2,2,False,5000.0,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",2018-07-02,2014-01-28,3018.000000,663.000000,1616.000000
3,3,False,9900.0,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",2016-11-28,2010-03-31,63078.000000,1746.000000,2434.000000
4,4,False,4800.0,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",2018-03-04,2012-07-30,8841.000000,523.000000,2043.000000
5,5,False,25800.0,RPG,"Single-player,Online Co-op,Local Co-op,Shared/...","RPG,Open World,Survival,Co-op,Fantasy,Online C...",NaT,2019-03-26,14519.939052,3012.805869,1006.817156
6,6,False,3600.0,"Action,Indie","Single-player,Steam Achievements,Steam Trading...","Great Soundtrack,Action,Violent,Indie,Top-Down...",2018-02-16,2012-10-23,39776.000000,1035.000000,1942.000000


In [7]:
all_data_drop = all_data.drop(['purchase_date','release_date'],axis =1)

In [8]:
#处理is_free
all_data_drop['is_free'] = np.multiply(all_data_drop['is_free'],1)

In [9]:
all_data_drop.head()

,id,is_free,price,genres,categories,tags,total_positive_reviews,total_negative_reviews,purchase_interval
0,0,0,3700.0,"Adventure,Casual,Indie","Single-player,Steam Trading Cards,Steam Cloud","Indie,Adventure,Story Rich,Casual,Atmospheric,...",372.0,96.0,1665.0
1,1,1,0.0,RPG,"Single-player,Partial Controller Support","Mod,Utilities,RPG,Game Development,Singleplaye...",23.0,0.0,472.0
2,2,0,5000.0,"Adventure,Casual,Indie","Single-player,Full controller support,Steam Tr...","Point & Click,Adventure,Story Rich,Comedy,Indi...",3018.0,663.0,1616.0
3,3,0,9900.0,"Action,RPG","Single-player,Multi-player,Steam Achievements,...","Medieval,RPG,Open World,Strategy,Sandbox,Actio...",63078.0,1746.0,2434.0
4,4,0,4800.0,"Action,Indie,Strategy","Single-player,Co-op,Steam Achievements,Full co...","Tower Defense,Co-op,Action,Strategy,Online Co-...",8841.0,523.0,2043.0


In [10]:
#处理genres
genres = all_data_drop['genres'].str.split(',')
genres = genres.str.join(sep=',').str.get_dummies(sep=',')
#处理categories
categories = all_data_drop['categories'].str.split(',')
categories = categories.str.join(sep=',').str.get_dummies(sep=',')
#处理tags
tags = all_data_drop['tags'].str.split(',')
tags = tags.str.join(sep=',').str.get_dummies(sep=',')

In [11]:
print(all_data_drop.shape)
print(tags.shape)
print(categories.shape)
print(genres.shape)

(447, 9)
(447, 317)
(447, 29)
(447, 20)


In [12]:
#组合one hot
all_concat = pd.concat([all_data_drop.reset_index(drop=True),genres.reset_index(drop=True),\
                          categories.reset_index(drop=True),tags.reset_index(drop=True)], axis=1)
all_concat.head()
print(all_concat.shape)

(447, 375)


In [13]:
all_concat = all_concat.drop(['genres','categories','tags'],axis = 1)
print(all_concat.shape)

(447, 372)


In [14]:
#查看有无重复项
duplicate_columns = all_concat.columns[all_concat.columns.duplicated()]
duplicate_columns

Index(['Action', 'Adventure', 'Animation & Modeling', 'Audio Production',
       'Casual', 'Co-op', 'Design & Illustration', 'Early Access',
       'Free to Play', 'Gore', 'Indie', 'Massively Multiplayer', 'Nudity',
       'RPG', 'Racing', 'Sexual Content', 'Simulation', 'Sports', 'Strategy',
       'Utilities', 'Violent'],
      dtype='object')

In [15]:
#drop重复项
#drop duplicates
all_concat = all_concat.loc[:,~all_concat.columns.duplicated()]
print(all_concat.shape)

(447, 351)


In [16]:
#观察各变量之间的关系

#correlation = train.iloc[:, 1:].corr().iloc[:, :]
#correlation
#correlation.describe()
all_concat = all_concat.drop(['is_free','price'],axis = 1)

In [17]:
#区分出测试集和训练集
test_x = all_concat[357:]
train_x = all_concat[:357]
y = train['playtime_forever']
train_x = train_x.drop(index=[197,269,275])
y = y.drop(index=[197,269,275])

In [18]:
#model----------------------------------------

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_x, y, test_size=0.2, random_state=0)

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score   #cross validation
from sklearn.model_selection import GridSearchCV      #调参
import warnings
warnings.filterwarnings("ignore")

In [21]:
#xgboost
import xgboost as xgb

xgr = xgb.XGBRegressor(           
                 #learning_rate=0.01,
                 #max_depth=16,
                 #min_child_weight=5,
                 n_estimators=500,                                                                    
                 #seed=42,
                 #num_boost_rounds=2000,
                 #objective="reg:squarederror",
                )

xgr.fit(train_x, y)

[14:17:18] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=500,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [22]:
accuracies = cross_val_score(estimator = xgr, X=train_x, y=y, cv=5, scoring='neg_mean_squared_error')
rmse_svr = np.sqrt(-accuracies).mean()
rmse_svr

[14:17:31] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:17:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:17:34] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:17:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[14:17:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


8.307060755769609

In [25]:
result_xgb = xgr.predict(data = test_x)

In [26]:
result_xgb = pd.DataFrame(result_xgb)
result_xgb['id']=range(len(result_xgb))
result_xgb.set_index('id',inplace=True)
result_xgb.columns = ['playtime_forever']
result_xgb.to_csv("./xgb.csv",index=True)